In [35]:
import os
import pandas as pd
from sqlalchemy import inspect
from sqlmodel import SQLModel, create_engine, Session, select
from urllib.parse import quote

from property_scraper.pvp.models.sale import Attachment

In [27]:
database_url = "sqlite:////media/emanuele/ai/property_scraper/demos/pvp.sqlite"
table = 'attachments'
column = 'attachment_link'
rootname = 'https://pvp-documenti.apps.pvc-os-caas01-rs.polostrategiconazionale.it'

In [3]:
engine = create_engine(database_url)

In [4]:
SQLModel.metadata.create_all(engine)

In [6]:
def get_columns_inspect(model_or_table):
    return [column.name for column in inspect(model_or_table).columns]

In [10]:
get_columns_inspect(Attachment)

['id',
 'attachment_id',
 'file_name',
 'attachment_link',
 'size',
 'description',
 'attachment_type_code',
 'is_lot_insertion',
 'content_base64',
 'asset_id']

In [41]:
with Session(engine) as session:
    statement = select(Attachment.attachment_link)
    results = session.exec(statement).all()
    urls = [os.path.join(rootname, quote(result[1:], safe=':/?=&')) for result in results]
    print('\n'.join(urls[:10]))
df = pd.DataFrame(urls)
df.to_csv('/media/emanuele/ai/property_scraper/demos/downloads/pvp/attachments/urls.csv', index=False)

https://pvp-documenti.apps.pvc-os-caas01-rs.polostrategiconazionale.it/allegati/99095/Perizia.pdf?versionId=adc6bd71-e423-4fd3-ada3-a7385369ad2b
https://pvp-documenti.apps.pvc-os-caas01-rs.polostrategiconazionale.it/allegati/99095/Avviso%20di%20vendita.pdf?versionId=80993e26-9bf8-4e6f-8915-f06730f2024a
https://pvp-documenti.apps.pvc-os-caas01-rs.polostrategiconazionale.it/allegati/99095/Ordinanza%20vendita.pdf?versionId=8e98d584-b327-45e3-b0b8-1ca2f917ba18
https://pvp-documenti.apps.pvc-os-caas01-rs.polostrategiconazionale.it/allegati/4240188/AVVISO%20DI%20VENDITA%20156-2010%20%282%29.pdf?versionId=ec9ae301-2199-4d06-b113-f49618639334
https://pvp-documenti.apps.pvc-os-caas01-rs.polostrategiconazionale.it/allegati/4240188/PERIZIA%20156-2010_compressed%20%281%29_Redacted.pdf?versionId=5d85a71c-47fa-48f6-8261-482a0d6dd2c3
https://pvp-documenti.apps.pvc-os-caas01-rs.polostrategiconazionale.it/allegati/4240188/DELEGA%20156-2010.pdf?versionId=7d1ed52f-cdd7-4dbc-8175-f43460f623df
https://pvp-